In [1]:
# import os
# os.chdir('../../')

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scr.util import *

In [3]:
import numpy as np
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import xgboost as xgb
import lightgbm as lgb
import catboost as cb
from sklearn.pipeline import Pipeline

In [49]:
df_train = pd.read_csv('data/null_survey/train_age_ok.csv')
df_test = pd.read_csv('data/null_survey/test_age_ok.csv')

In [50]:
df_train.isna().sum()

id                          0
Age                         0
TypeofContact               0
CityTier                    0
DurationOfPitch           121
Occupation                  0
Gender                      0
NumberOfPersonVisiting      0
NumberOfFollowups           0
ProductPitched              0
PreferredPropertyStar       0
NumberOfTrips              22
Passport                    0
PitchSatisfactionScore      0
Designation                 0
MonthlyIncome              56
ProdTaken                   0
Marry                       0
Car                         0
Child                       0
AgeGroup                    0
dtype: int64

In [51]:
df_test.isna().sum()

id                          0
Age                         0
TypeofContact               0
CityTier                    0
DurationOfPitch           131
Occupation                  0
Gender                      0
NumberOfPersonVisiting      0
NumberOfFollowups           0
ProductPitched              0
PreferredPropertyStar       0
NumberOfTrips              42
Passport                    0
PitchSatisfactionScore      0
Designation                 0
MonthlyIncome              49
Marry                       0
Car                         0
Child                       0
AgeGroup                    0
dtype: int64

# 今回調査する欠損値：Age, DurationOfPitch, NumberOfTrips, *MonthlyIncome*

# MonthlyIncome

In [52]:
df_all = pd.concat([df_train, df_test], axis=0)

In [53]:
df_all.columns

Index(['id', 'Age', 'TypeofContact', 'CityTier', 'DurationOfPitch',
       'Occupation', 'Gender', 'NumberOfPersonVisiting', 'NumberOfFollowups',
       'ProductPitched', 'PreferredPropertyStar', 'NumberOfTrips', 'Passport',
       'PitchSatisfactionScore', 'Designation', 'MonthlyIncome', 'ProdTaken',
       'Marry', 'Car', 'Child', 'AgeGroup'],
      dtype='object')

In [54]:
# MonthlyIncomeを予測するための特徴量を選択
predict_income_features = ['Age', 'TypeofContact','CityTier','Occupation','Gender','NumberOfPersonVisiting',
                        'NumberOfFollowups','ProductPitched','PreferredPropertyStar','PitchSatisfactionScore', 'Passport','Designation' ,'Marry', 'Car','Child', 'AgeGroup']

# 主キー用idを取得
id_train = df_train.loc[df_train['MonthlyIncome'].isna()].index
id_test = df_test.loc[df_test['MonthlyIncome'].isna()].index

# 
X_train_predict_income = df_all.loc[~(df_all['MonthlyIncome'].isna()), predict_income_features]
X_test_predict_income = df_all.loc[df_all['MonthlyIncome'].isna(), predict_income_features]
y_train_predict_income = df_all.loc[~(df_all['MonthlyIncome'].isna()), 'MonthlyIncome']
y_test_predict_income = None

In [55]:
def mapping_first_category_plus_agegroup(df_train, df_test):

    mapping = {
        'No': 0,
        'Self Enquiry': 1,
        'Company Invited': 2
    }
    df_train.loc[:, 'TypeofContact'] = df_train.loc[:, 'TypeofContact'].map(mapping)
    df_test.loc[:, 'TypeofContact'] = df_test.loc[:, 'TypeofContact'].map(mapping)

    mapping = {
        'Salaried': 0,
        'Small Business': 1,
        'Large Business': 2
    }
    df_train.loc[:, 'Occupation'] = df_train.loc[:, 'Occupation'].map(mapping)
    df_test.loc[:, 'Occupation'] = df_test.loc[:, 'Occupation'].map(mapping)

    mapping = {
        'male': 0,
        'female': 1
    }
    df_train.loc[:, 'Gender'] = df_train.loc[:, 'Gender'].map(mapping)
    df_test.loc[:, 'Gender'] = df_test.loc[:, 'Gender'].map(mapping)

    mapping = {
        'Super Deluxe': 0,
        'Standard': 1,
        'King': 2,
        'Deluxe': 3,
        'Basic': 4
    }
    df_train.loc[:, 'ProductPitched'] = df_train.loc[:, 'ProductPitched'].map(mapping)
    df_test.loc[:, 'ProductPitched'] = df_test.loc[:, 'ProductPitched'].map(mapping)

    mapping = {
        'Manager': 0,
        'VP': 1,
        'AVP': 2,
        'Senior Manager': 3,
        'Executive': 4
    }

    df_train.loc[:, 'Designation'] = df_train.loc[:, 'Designation'].map(mapping)
    df_test.loc[:, 'Designation'] = df_test.loc[:, 'Designation'].map(mapping)

    mapping = {
        'Married': 0,
        'Single': 1,
        'Divorced': 2,
    }
    df_train.loc[:, 'Marry'] = df_train.loc[:, 'Marry'].map(mapping)
    df_test.loc[:, 'Marry'] = df_test.loc[:, 'Marry'].map(mapping)

    mapping = {
        'No Car': 0,
        'Has Car': 1,
    }
    df_train.loc[:, 'Car'] = df_train.loc[:, 'Car'].map(mapping)
    df_test.loc[:, 'Car'] = df_test.loc[:, 'Car'].map(mapping)

    mapping = {
        '0_child': 0,
        '1_child': 1,
        '2_child': 2,
        '3_child': 3
    }
    df_train.loc[:, 'Child'] = df_train.loc[:, 'Child'].map(mapping)
    df_test.loc[:, 'Child'] = df_test.loc[:, 'Child'].map(mapping)
    
    mapping = {
        '10s': 1,
        '20s': 2,
        '30s': 3,
        '40s': 4,
        '50s': 5,
        '60s': 6
    }
    df_train.loc[:, 'AgeGroup'] = df_train.loc[:, 'AgeGroup'].map(mapping)
    df_test.loc[:, 'AgeGroup'] = df_test.loc[:, 'AgeGroup'].map(mapping)
    return df_train, df_test

In [56]:
X_train_predict_income, X_test_predict_income = mapping_first_category_plus_agegroup(X_train_predict_income, X_test_predict_income)

In [57]:
X_train_predict_income, X_test_predict_income = feature_to_int(X_train_predict_income, X_test_predict_income)

In [58]:
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
cv_list = list(skf.split(X_train_predict_income, y_train_predict_income))

def train_catboost(X, y, cv, params: dict = None):
    if params is None:
        params = {}

    models = []
    n_records = len(X)
    oof_pred = np.zeros((n_records, ))
    for i, (tr_idx, va_idx) in enumerate(cv):
        tr_x, va_x = X.iloc[tr_idx], X.iloc[va_idx]
        tr_y, va_y = y.iloc[tr_idx], y.iloc[va_idx]
        model = cb.CatBoostRegressor(**params)
        model.fit(tr_x, tr_y, eval_set=(va_x, va_y), use_best_model=True, verbose=100)
        oof_pred[va_idx] = model.predict(va_x)
        models.append(model)
    return oof_pred, models

params = {
    'loss_function': 'RMSE',
    'eval_metric': 'RMSE',
    'iterations': 10000,
    'learning_rate': 0.005,
    'depth': 5,
    'l2_leaf_reg': 5,    # 3 ~ 10
    'verbose': 200,
    'random_seed': 42,
}

oof, models = train_catboost(X_train_predict_income, y_train_predict_income, cv_list, params)

c:\Users\ricke\anaconda3\envs\competition\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


0:	learn: 75968.1945889	test: 75751.7708231	best: 75751.7708231 (0)	total: 2.97ms	remaining: 29.7s
100:	learn: 60414.0176700	test: 59713.2876622	best: 59713.2876622 (100)	total: 131ms	remaining: 12.8s
200:	learn: 52880.7908840	test: 51838.6739064	best: 51838.6739064 (200)	total: 252ms	remaining: 12.3s
300:	learn: 49420.9266595	test: 48265.2731952	best: 48265.2731952 (300)	total: 362ms	remaining: 11.7s
400:	learn: 47814.6925050	test: 46731.7941896	best: 46731.7941896 (400)	total: 472ms	remaining: 11.3s
500:	learn: 46928.8052311	test: 45923.4029043	best: 45923.4029043 (500)	total: 580ms	remaining: 11s
600:	learn: 46398.8680948	test: 45513.8699524	best: 45513.8699524 (600)	total: 688ms	remaining: 10.8s
700:	learn: 46038.0894020	test: 45270.3483914	best: 45270.3483914 (700)	total: 796ms	remaining: 10.6s
800:	learn: 45764.3436492	test: 45114.9931486	best: 45114.9931486 (800)	total: 907ms	remaining: 10.4s
900:	learn: 45550.4882078	test: 45017.7603524	best: 45017.7603524 (900)	total: 1.02s	re

In [60]:
y_test_predict_income = np.zeros(len(X_test_predict_income))

# アンサンブル
for model in models:
    y_test_predict_income += model.predict(X_test_predict_income) / len(models)

y_test_predict_income = pd.DataFrame({
    'MonthlyIncome': y_test_predict_income
})

In [61]:
# 訓練データとテストデータに分割する
predicted_train_age = y_test_predict_income.iloc[:len(id_train), 0]
predicted_test_age = y_test_predict_income.iloc[len(id_train): , 0]

# id(主キー)を与える。
predicted_train_age.index = id_train
predicted_test_age.index = id_test

In [67]:
df_train[2550:2560]

,id,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisiting,NumberOfFollowups,ProductPitched,...,NumberOfTrips,Passport,PitchSatisfactionScore,Designation,MonthlyIncome,ProdTaken,Marry,Car,Child,AgeGroup
2550,2550,52,Self Enquiry,1,720.0,Small Business,male,3.0,4.0,Basic,...,4.0,0,2,Executive,300000.0,1,Single,No Car,1_child,50s
2551,2551,35,Company Invited,1,960.0,Small Business,female,3.0,4.0,Basic,...,2.0,0,2,Executive,NaN,0,Divorced,Has Car,0_child,30s
2552,2552,34,Company Invited,1,1140.0,Large Business,male,3.0,3.0,Basic,...,5.0,0,2,Executive,310000.0,1,Married,No Car,1_child,30s
2553,2553,44,Self Enquiry,2,360.0,Salaried,male,3.0,4.0,Deluxe,...,5.0,1,2,AVP,457830.0,0,Divorced,Has Car,2_child,40s
2554,2554,36,Company Invited,2,780.0,Small Business,male,3.0,4.0,Standard,...,2.0,1,2,Senior Manager,363810.0,1,Married,No Car,2_child,30s
2555,2555,24,Company Invited,2,480.0,Salaried,female,2.0,4.0,Standard,...,2.0,1,1,Senior Manager,415215.0,1,Married,No Car,1_child,20s
2556,2556,37,Self Enquiry,2,1440.0,Salaried,male,3.0,4.0,Basic,...,28.0,1,2,Executive,300000.0,1,Single,No Car,2_child,30s
2557,2557,39,Company Invited,1,NaN,Salaried,male,2.0,3.0,Deluxe,...,1.0,0,4,Manager,NaN,0,Married,No Car,1_child,30s
2558,2558,31,Self Enquiry,2,1920.0,Small Business,female,3.0,3.0,Deluxe,...,2.0,0,1,Manager,325110.0,1,Single,No Car,2_child,30s
2559,2559,48,Self Enquiry,1,480.0,Large Business,female,3.0,5.0,Standard,...,6.0,0,2,Senior Manager,374805.0,0,Married,No Car,2_child,40s


In [69]:
df_test[2150:2160]

,id,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisiting,NumberOfFollowups,ProductPitched,PreferredPropertyStar,NumberOfTrips,Passport,PitchSatisfactionScore,Designation,MonthlyIncome,Marry,Car,Child,AgeGroup
2150,5639,48,Self Enquiry,3,1740.0,Small Business,male,3.0,4.0,Deluxe,3.0,5.0,0,3,Manager,378975.0,Divorced,No Car,1_child,40s
2151,5640,25,Self Enquiry,2,900.0,Small Business,male,3.0,4.0,Basic,3.0,2.0,0,3,Executive,258660.0,Single,Has Car,1_child,20s
2152,5641,46,Self Enquiry,3,900.0,Small Business,female,3.0,4.0,Basic,3.0,5.0,0,3,Executive,310000.0,Divorced,Has Car,2_child,40s
2153,5642,44,Self Enquiry,2,960.0,Small Business,male,3.0,4.0,King,3.0,NaN,0,3,VP,441810.0,Married,Has Car,1_child,40s
2154,5643,20,Self Enquiry,2,1140.0,Small Business,male,3.0,4.0,Basic,3.0,2.0,0,4,Executive,298950.0,Married,Has Car,1_child,20s
2155,5644,30,Self Enquiry,2,420.0,Large Business,male,3.0,4.0,Standard,3.0,12.0,0,4,Senior Manager,353985.0,Divorced,Has Car,1_child,30s
2156,5645,25,Self Enquiry,2,900.0,Salaried,male,3.0,4.0,Basic,3.0,3.0,0,4,Executive,312030.0,Single,Has Car,2_child,20s
2157,5646,30,Company Invited,2,1020.0,Small Business,male,2.0,3.0,Basic,3.0,8.0,0,4,Executive,NaN,Single,No Car,1_child,30s
2158,5647,35,Self Enquiry,1,540.0,Small Business,male,3.0,4.0,Basic,3.0,2.0,0,4,Executive,NaN,Married,Has Car,1_child,30s
2159,5648,24,Self Enquiry,1,480.0,Small Business,female,4.0,5.0,Deluxe,4.0,3.0,0,1,Executive,371010.0,Married,No Car,2_child,20s


In [70]:
# 欠損値を予測値で補完する
df_train.loc[df_train['MonthlyIncome'].isna(), 'MonthlyIncome'] = round(predicted_train_age)
df_test.loc[df_test['MonthlyIncome'].isna(), 'MonthlyIncome'] = round(predicted_test_age)

In [71]:
df_train[2550:2560]

,id,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisiting,NumberOfFollowups,ProductPitched,...,NumberOfTrips,Passport,PitchSatisfactionScore,Designation,MonthlyIncome,ProdTaken,Marry,Car,Child,AgeGroup
2550,2550,52,Self Enquiry,1,720.0,Small Business,male,3.0,4.0,Basic,...,4.0,0,2,Executive,300000.0,1,Single,No Car,1_child,50s
2551,2551,35,Company Invited,1,960.0,Small Business,female,3.0,4.0,Basic,...,2.0,0,2,Executive,293944.0,0,Divorced,Has Car,0_child,30s
2552,2552,34,Company Invited,1,1140.0,Large Business,male,3.0,3.0,Basic,...,5.0,0,2,Executive,310000.0,1,Married,No Car,1_child,30s
2553,2553,44,Self Enquiry,2,360.0,Salaried,male,3.0,4.0,Deluxe,...,5.0,1,2,AVP,457830.0,0,Divorced,Has Car,2_child,40s
2554,2554,36,Company Invited,2,780.0,Small Business,male,3.0,4.0,Standard,...,2.0,1,2,Senior Manager,363810.0,1,Married,No Car,2_child,30s
2555,2555,24,Company Invited,2,480.0,Salaried,female,2.0,4.0,Standard,...,2.0,1,1,Senior Manager,415215.0,1,Married,No Car,1_child,20s
2556,2556,37,Self Enquiry,2,1440.0,Salaried,male,3.0,4.0,Basic,...,28.0,1,2,Executive,300000.0,1,Single,No Car,2_child,30s
2557,2557,39,Company Invited,1,NaN,Salaried,male,2.0,3.0,Deluxe,...,1.0,0,4,Manager,313747.0,0,Married,No Car,1_child,30s
2558,2558,31,Self Enquiry,2,1920.0,Small Business,female,3.0,3.0,Deluxe,...,2.0,0,1,Manager,325110.0,1,Single,No Car,2_child,30s
2559,2559,48,Self Enquiry,1,480.0,Large Business,female,3.0,5.0,Standard,...,6.0,0,2,Senior Manager,374805.0,0,Married,No Car,2_child,40s


In [72]:
df_test[2150:2160]

,id,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisiting,NumberOfFollowups,ProductPitched,PreferredPropertyStar,NumberOfTrips,Passport,PitchSatisfactionScore,Designation,MonthlyIncome,Marry,Car,Child,AgeGroup
2150,5639,48,Self Enquiry,3,1740.0,Small Business,male,3.0,4.0,Deluxe,3.0,5.0,0,3,Manager,378975.0,Divorced,No Car,1_child,40s
2151,5640,25,Self Enquiry,2,900.0,Small Business,male,3.0,4.0,Basic,3.0,2.0,0,3,Executive,258660.0,Single,Has Car,1_child,20s
2152,5641,46,Self Enquiry,3,900.0,Small Business,female,3.0,4.0,Basic,3.0,5.0,0,3,Executive,310000.0,Divorced,Has Car,2_child,40s
2153,5642,44,Self Enquiry,2,960.0,Small Business,male,3.0,4.0,King,3.0,NaN,0,3,VP,441810.0,Married,Has Car,1_child,40s
2154,5643,20,Self Enquiry,2,1140.0,Small Business,male,3.0,4.0,Basic,3.0,2.0,0,4,Executive,298950.0,Married,Has Car,1_child,20s
2155,5644,30,Self Enquiry,2,420.0,Large Business,male,3.0,4.0,Standard,3.0,12.0,0,4,Senior Manager,353985.0,Divorced,Has Car,1_child,30s
2156,5645,25,Self Enquiry,2,900.0,Salaried,male,3.0,4.0,Basic,3.0,3.0,0,4,Executive,312030.0,Single,Has Car,2_child,20s
2157,5646,30,Company Invited,2,1020.0,Small Business,male,2.0,3.0,Basic,3.0,8.0,0,4,Executive,276628.0,Single,No Car,1_child,30s
2158,5647,35,Self Enquiry,1,540.0,Small Business,male,3.0,4.0,Basic,3.0,2.0,0,4,Executive,315087.0,Married,Has Car,1_child,30s
2159,5648,24,Self Enquiry,1,480.0,Small Business,female,4.0,5.0,Deluxe,4.0,3.0,0,1,Executive,371010.0,Married,No Car,2_child,20s


In [73]:
df_train.isna().sum()

id                          0
Age                         0
TypeofContact               0
CityTier                    0
DurationOfPitch           121
Occupation                  0
Gender                      0
NumberOfPersonVisiting      0
NumberOfFollowups           0
ProductPitched              0
PreferredPropertyStar       0
NumberOfTrips              22
Passport                    0
PitchSatisfactionScore      0
Designation                 0
MonthlyIncome               0
ProdTaken                   0
Marry                       0
Car                         0
Child                       0
AgeGroup                    0
dtype: int64

In [74]:
df_test.isna().sum()

id                          0
Age                         0
TypeofContact               0
CityTier                    0
DurationOfPitch           131
Occupation                  0
Gender                      0
NumberOfPersonVisiting      0
NumberOfFollowups           0
ProductPitched              0
PreferredPropertyStar       0
NumberOfTrips              42
Passport                    0
PitchSatisfactionScore      0
Designation                 0
MonthlyIncome               0
Marry                       0
Car                         0
Child                       0
AgeGroup                    0
dtype: int64

In [75]:
df_train.to_csv('data/null_survey/train_age_income_ok.csv', index=False)
df_test.to_csv('data/null_survey/test_age_income_ok.csv', index=False)